# Preview Evaluation - Perplexity

Thia is a notebook to make a preview comparison of the generations when using LLM JUDGE

In [2]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
from utils.metrics.calculate_metric import calculate_agg_metric
import os
import numpy as np
import json

/home/users/caio.rhoden/miniconda3/envs/nq/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 10-31 15:30:01 [__init__.py:220] No platform detected, vLLM is running on UnspecifiedPlatform
WARNING 10-31 15:30:02 [_custom_ops.py:20] Failed to import from vllm._C with ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')


In [3]:
exp_names = ["vllm3", "vllm5", "batch-hf3", "batch-hf5", "single-hf"]

## Calculate Metrics and save them

In [ ]:


questions_path = "../../questions_2500_42_dev.feather"
for exp in exp_names:
    for file in os.listdir(f"../{exp}/generations"):
        if f"result_{exp}.feather" not in os.listdir("."):
            metrics = calculate_agg_metric(
                metrics=["rouge_l"],
                generation_path=f"../{exp}/generations/{file}",
                reference_path=questions_path,
                saving_path=f"result_{exp}.feather"
            )




Could not memory_map compressed IPC file, defaulting to normal read. Toggle off 'memory_map' to silence this warning.


## Load Data

In [8]:
results = []
for f in os.listdir("."):
    if f.endswith("feather"):
        processsed_namefile = f.split(".")[0].split("result_")[1]
        results.append(
            pl.read_ipc(f, memory_map=False)
            .with_columns(
                pl.lit(f"{processsed_namefile}").alias("exp_name"),
            )
        )

results = pl.concat(results)
with pl.Config(
    tbl_rows=-1,             # Show all rows (-1 means all)
    tbl_cols=-1,             # Show all columns (-1 means all)
    fmt_str_lengths=100    # Set string column width
):
    print(results.group_by("exp_name").agg(pl.col("mean").mean()).sort("mean", descending=True))

shape: (5, 2)
┌───────────┬──────────┐
│ exp_name  ┆ mean     │
│ ---       ┆ ---      │
│ str       ┆ f64      │
╞═══════════╪══════════╡
│ single-hf ┆ 0.109554 │
│ batch-hf5 ┆ 0.070701 │
│ batch-hf3 ┆ 0.070684 │
│ vllm5     ┆ 0.070252 │
│ vllm3     ┆ 0.069319 │
└───────────┴──────────┘


In [9]:
results.filter(pl.col("mean") > 0).group_by("exp_name").agg(pl.col("mean").count())

exp_name,mean
str,u32
"""batch-hf5""",644
"""batch-hf3""",668
"""vllm5""",653
"""single-hf""",44
"""vllm3""",578
